## 📂 Data Cube with Random Numbers
To create a cubic dataset with random numbers, we will import numpy, xarray, zarr, and pandas libraries. We will generate 720 latitudes from -90 to +90, 1440 longitudes from -180 to +180, and 365 days from 2023–01–01 to 2023–12–31. Next, we will create a 3D array with random numbers ranging from -20 to 30 and set time, latitude, and longitude. After that, we will convert the array to Xarray, save it in Zarr format, and label the data as "air temperature". The following lines do these steps for you:

In [ ]:
import numpy as np
import xarray as xr
import zarr
import pandas as pd

# Set dimensions
lat = np.linspace(-90, 90, 720)
lon = np.linspace(-180, 180, 1440)
time = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')

# Create 3D array with random numbers
data = np.random.uniform(low=-20, high=30, size=(len(time), len(lat), len(lon)))

# Create xarray DataArray with coordinate labels
data_array = xr.DataArray(data, coords={"time": time, "lat": lat, "lon": lon}, dims=["time", "lat", "lon"])

# Save DataArray as Zarr file
data_array.to_dataset(name="air_temperature").to_zarr("random_data.zarr", mode="w")

You can open this data cube with these lines:

In [ ]:
ds = xr.open_dataset("random_data.zarr", chunks={}, engine="zarr")
da = ds["air_temperature"][:,:,:]
da

If the data cube is created correctly, you will see the variable name (in this case, air temperature) and dimensions for time (365), latitude (720), and longitude (1440). In the next section, we will use this data cube to plot it with Lexcube. But before that, let’s create a data cube with the climate data.

## 🗂 ️ Data Cube with Climate Data

To create the data cube with climate data, we will use the GeoTIFF files generated in this post (refer to sections 📥 Download Climate Data and 🔄 Convert Climate Data to GeoTIFF). You should have the mean air temperature of the US for July from 1981 to 2020 in GeoTIFF format in your content folder. Here, we are going to get a list of paths for those GeoTIFF files to read them in a loop :

In [ ]:
import os
import glob

directory = '/content/'
extension = '.tif'

# Get a list of all GeoTIFF files in the directory
geotiff_files = glob.glob(os.path.join(directory, '*' + extension))

geotiff_files.sort()
# Print the list of GeoTIFF files
print(geotiff_files)

Before reading the raster files in a loop, we need to create a list of date ranges for those GeoTIFF files, as the GeoTIFF files cover July from 1981 to 2020. Next, we will read each raster file using the rasterio package, convert the raster file to Xarray with latitude and longitude extracted from each file, and create a data cube by appending the 2D data from each raster file. Finally, we’ll set the time in the data cube based on the time range we generated and save it as a Zarr file:

In [ ]:
import rasterio
import xarray as xr
import zarr

# Create a time list
time = pd.date_range(start='1981-07-01', end='2021-07-01', freq='Y')

# Read raster files and create a list of xarray DataArrays
data_arrays = []
for file in geotiff_files:
    with rasterio.open(file) as src:
        data = src.read(1)  # Read the first band
        height, width = src.shape
        y_values = np.arange(height) * src.transform[4] + src.transform[5]
        x_values = np.arange(width) * src.transform[0] + src.transform[2]
        da = xr.DataArray(data, dims=("y", "x"), coords={"y": y_values, "x": x_values}, name="air_temperature")
        data_arrays.append(da)

# Combine DataArrays into a single xarray Dataset
ds = xr.concat(data_arrays, dim="time")
ds["time"] = ("time", time)  # Assign month numbers as time coordinates

# Save the xarray Dataset as a zarr file
ds.to_zarr("climate.zarr", mode="w")

Similar to the previous section, we can read the saved Zarr file using the following:

In [ ]:
ds = xr.open_dataset("climate.zarr", chunks={}, engine="zarr")
da = ds["air_temperature"][:,:,:]
da

If the file is saved correctly, you should see the Xarray details (40 layers in time, 1405 grids on the x-axis, and 621 grids on the y-axis).

## 🌍 3D Visualization of Xarray by Lexcube
Now that we have two Zarr files (one based on random numbers and the second based on climate data), we are ready to plot the 3D visualization of these two data cubes. We will begin by installing the Lexcube library:

In [ ]:
pip install lexcube

Let’s start with the first data cube (random numbers):

In [ ]:
import xarray as xr
import lexcube
ds = xr.open_dataset("random_data.zarr", chunks={}, engine="zarr")
da = ds["air_temperature"][:,:,:]
w = lexcube.Cube3DWidget(da,cmap="thermal_r", vmin=-20, vmax=30)
w

Since the first data cube has been generated based on random numbers, you will see a completely noisy image in all three dimensions. However, it would be a good exercise to demonstrate how to convert any data cube to Xarray for use in Lexcube. To have a better and more meaningful 3D plot, let’s visualize the second data cube (climate data). We will replicate the same lines but with the new Zarr file name, which is ‘climate.zarr’:

In [ ]:
ds = xr.open_dataset("climate.zarr", chunks={}, engine="zarr")
da = ds["air_temperature"][:,:,:]

w = lexcube.Cube3DWidget(da,cmap="RdYlBu_r", vmin=0, vmax=30)
w

Let’s zoom in on Central Valley in California. You can see the mean air temperature for July 2020 displayed on the top layer. Additionally, by hovering your mouse over the latitude and longitude axes, you can see the air temperature for different years. Which year was the warmest? At which coordinates?

## 📦 What Else Can We Do with Lexcube
Let’s assume you want to clip this plot for a specific location (latitude and longitude) and a specific time. Instead of manually zooming in and out on each axis, which is not very convenient, you can activate the slider by running the following:

In [ ]:
w.show_sliders()

with that slider, you can clip your plot for any location and timeframe:

And last but not least, if you want to save the plot in your local folder, you can do so by running the following:

In [ ]:
w.savefig(fname="climate.png", include_ui=True, dpi_scale=2.0)

The PNG file will be saved in your folder. If you want to change the color ramp, Lexcube supports many color maps that you can find in the GitHub repository mentioned in the reference section. Also, if you want to plot other geospatial datasets with Lexcube, check out the following post for downloading satellite and geospatial databases for a specific location and times without writing any code:

[How to Download Satellite Images Without Writing a Single Line of Code](https://medium.com/@mahyar.aboutalebi/how-to-download-satellite-images-without-writing-a-single-line-of-code-c08b1f910203)

## 📝 Conclusion
Data visualization nowadays is a great skill that can help us tell stories and efficiently present the outcomes in our reports. Lexcube is one of the libraries in this field that helps us visualize data cubes, particularly 3D geospatial databases, quickly and in a visually engaging way. In this post, we cover how to prepare your datasets for this library and how to use it for your data visualization projects. Specifically, we created two datasets: one based on completely random numbers and the second based on air temperature reported by climate databases. Additional sources are provided if you want to create your data cube for any location and time frame.

## 📚 References
M., Söchting, M. D., Mahecha, D., Montero, and G., Scheuermann, Lexcube: Interactive Visualization of Large Earth System Data Cubes (2023). IEEE Computer Graphics and Applications. doi:10.1109/MCG.2023.3321989.

[https://github.com/msoechting/lexcube](https://github.com/msoechting/lexcube)

[https://pubmed.ncbi.nlm.nih.gov/37812545/](https://pubmed.ncbi.nlm.nih.gov/37812545/)

[https://eo4society.esa.int/2022/05/25/exploring-earth-system-data-with-lexcube/](https://eo4society.esa.int/2022/05/25/exploring-earth-system-data-with-lexcube/)

[https://www.linkedin.com/posts/miguel-mahecha-625548197_lexcube-activity-7156200204700966913-5lOb?utm_source=share&utm_medium=member_desktop](https://www.linkedin.com/posts/miguel-mahecha-625548197_lexcube-activity-7156200204700966913-5lOb?utm_source=share&utm_medium=member_desktop)

PRISM Climate Group, Oregon State University, [https://prism.oregonstate.edu](https://prism.oregonstate.edu), date created 1981–2022, accessed 19 Dec 2024.